In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
import os


### Notebook to create and test functions for webscraping

##### Using [Wikipedia to get current list of S&P 500 companies](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies)

In [ ]:
def get_content():
    wiki_url = r'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    payload = requests.get(wiki_url)
    soup = BeautifulSoup(payload.content)
    return(soup)


In [ ]:
s = get_content()

In [ ]:
def get_constituents(soup, name):
    table = soup.find('table', id=name)
    header = []
    rows = []
    for i, row in enumerate(table.find_all('tr')):
        if i == 0:
            header = [el.text.strip() for el in row.find_all('th')]
        else:
            rows.append([el.text.strip() for el in row.find_all('td')])
    
    df = pd.DataFrame(rows, columns = header)
    return(df)


In [ ]:
constituents = get_constituents(s, 'constituents')
base = constituents[['Symbol', 'Date added']].rename(columns ={ 'Symbol':'ticker', 'Date added':'date_added'}).sort_values(by = 'ticker').reset_index(drop = True)
base['date_added'] = pd.to_datetime(base['date_added'])
base['currently_listed'] = True
base.head()

In [ ]:
rows = []
change_html = s.find('table', id='changes')
for i, row in enumerate(change_html.find_all('tr')):
    if i > 1:
        rows.append([el.text.strip() for el in row.find_all('td')])
        
df = pd.DataFrame(rows, columns = ['Date', 'ADD_Symbol', 'ADD_Security', 'RMV_Symbol', 'RMV_Security', 'Reason'])       
df['Date'] = pd.to_datetime(df['Date'])
df = df.replace('', None)

#Get Most recent add dates per ticker
add_dates = df.loc[df['ADD_Symbol'].isnull() == False][['Date', 'ADD_Symbol']].\
    groupby('ADD_Symbol').agg({'Date':'max'}).reset_index().\
    rename(columns = {'ADD_Symbol':'ticker', 'Date':'date_added'})
#Get Most recent remove dates per ticker
rmv_dates = df.loc[df['RMV_Symbol'].isnull() == False][['Date', 'RMV_Symbol']].\
    groupby('RMV_Symbol').agg({'Date':'max'}).reset_index().\
    rename(columns = {'RMV_Symbol':'ticker', 'Date':'date_removed'})

all_tickers = add_dates.merge(rmv_dates, on = 'ticker', how = 'outer')

In [ ]:
tickers_total = base.merge(all_tickers, on = ['ticker', 'date_added'], how = 'outer')
tickers_total['currently_listed'] = tickers_total['currently_listed'].fillna(False)
#Classify anything without a date added as 1957-01-01
tickers_total['date_added'] = [pd.to_datetime('1957-01-01') if pd.isnull(d) == True else d for d in tickers_total['date_added']]

dups = list(tickers_total.groupby('ticker').size().loc[lambda x: x > 1].index)
no_dups = tickers_total.loc[~tickers_total['ticker'].isin(dups)].sort_values(by = ['ticker', 'date_added'])
fix_dups = tickers_total.loc[tickers_total['ticker'].isin(dups)].sort_values(by = ['ticker', 'date_added'])
fixed_dups = fix_dups.groupby('ticker').agg({'date_added':'max', 'currently_listed':'max'}).reset_index()
fixed_dups['date_removed'] = pd.NaT
order = ['ticker', 'date_added', 'date_removed', 'currently_listed']

fixed_dups = fixed_dups[order]
no_dups = no_dups[order]

tickers_total = pd.concat([fixed_dups, no_dups], ignore_index = True).sort_values(by = ['date_added', 'ticker']).reset_index(drop = True)


#Create an int ticker_id that will be used as a primary key for tickers
tickers_total['ticker_id'] = [i + 10000 for i in tickers_total.index]

col_order = [
    'ticker_id', 'ticker', 'date_added', 'date_removed', 'currently_listed'
]
tickers_total = tickers_total[col_order]

tickers_total.head()

In [ ]:
#write to db
conn = os.getenv('STOCK_DB_CONN')
engine = create_engine(conn)
tickers_total.to_sql(name = 's_and_p_500_history', con = engine, schema = 'NASDAQ', if_exists= 'replace', index = False, method = 'multi')
